<a href="https://colab.research.google.com/github/vanessadcamara/nlp-colab/blob/main/extrator_de_aspectos_vanessa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carregando Pacotes

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
!pip install --upgrade spacy
!python -m spacy
!python -m spacy download pt_core_news_lg
!pip install senticnet
!pip install tqdm
!pip install pandas
!python setup.py install
!python -m spacy download en_core_web_lg
!python -m spacy download pt_core_news_sm

Usage: python -m spacy [OPTIONS] COMMAND [ARGS]...
Try 'python -m spacy --help' for help.

Error: Missing command.
     |████████████████████████████████| 568.2 MB 7.0 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
python3: can't open file 'setup.py': [Errno 2] No such file or directory
     |████████████████████████████████| 400.7 MB 6.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
     |████████████████████████████████| 13.0 MB 25.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


## Importando Bibliotecas

In [61]:
!pip -m spacy download pt_core_news_lg
import pt_core_news_sm
import spacy
from spacy import displacy
from senticnet.senticnet import SenticNet
from senticnet.babelsenticnet import BabelSenticNet
import xml.etree.ElementTree as ET
import pandas as pd
import pathlib
import re
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from enum import Enum
import pandas as pd


Usage:   
  pip3 <command> [options]

no such option: -m


## Exemplo de Analisador de Dependências

In [62]:
nlp = pt_core_news_sm.load()
sentence = "o livro é muito estranho"
doc = nlp(sentence)
aspectos = []
bsn = BabelSenticNet('pt') 
for token in doc:
    print(token.text, token.tag_, token.head.text, token.dep_, token.pos_)
display = displacy.render(doc, style='dep', jupyter=True)
display

o DET livro det DET
livro NOUN estranho nsubj NOUN
é AUX estranho cop AUX
muito ADV estranho advmod ADV
estranho ADJ estranho ROOT ADJ


## Funções Auxiliares 

Referências: [3].

* essas funções modularizam a implementação de trechos de código comuns para algumas regras
* a função OP foi implementada para extrair palavras de opinião de um arquivo [3].


In [63]:
def relation_nsubj(doc):
    aux1 = " "
    aux2 = " "
    for i in doc:
        if i.dep_ == "nsubj":
            aux1 = i.text
            aux2 = i.head.text
            break;
    return aux1, aux2

def is_auxiliar(doc):
    auxiliar = 0    
    for i in doc:
        if i.pos_ == "AUX":
            auxiliar = 1
            break
    return auxiliar

def OP():
    arq = open("/content/drive/MyDrive/Vanessa/datasets/palavras.txt")
    op = set(line.strip() for line in arq)
    arq.close()
    return op
op = OP()

## Regra 1

References: [1].

0, exemplo: O **livro** é *muito estranho*.

> Trigger: quando o token ativo é considerado o sujeito sintático de um token.
Comportamento: se um token ativo h está em uma relação de substantivo com uma palavra t, então:
1. se t tem qualquer modificador adverbial ou adjetivo e o modificador existe no SenticNet, então t é extraído
como um aspecto.

In [64]:
def regra1(doc,aspectos):   
    token_h = ""
    token_t = ""
    for i in doc:
        if i.dep_ == "nsubj" or i.dep_ == "nsubj:pass":
            token_h = i
            token_t = i.head.text
            break; 
    for i in doc:
        if i.head.text == token_t:
            if i.dep_ == "advmod" or i.dep_ == "adjmod":
                try:
                    concept = bsn.concept(i.text)
                except Exception as e:
                    return 
                finally:
                    aspectos.append(token_t)
    return aspectos

## Regra 2
Referências: [1].

1, exemplo: A **bateria** *dura pouco*.

> se a frase não tem verbo auxiliar, ou seja, é, foi, iria, deveria, poderia, então: • se o verbo t é modificado por um adjetivo ou advérbio ou está em relação modificadora de oração adverbial com outro token, então ambos he t são extraídos como aspectos. Em (1), bateria está em uma relação de sujeito com dura e dura é pouco modificada pelo adjetivo modificador, portanto tanto os aspectos último quanto bateria são extraídos. 
 (1) A bateria dura pouco.

In [65]:
def regra2(doc, aspectos):
    token_h = relation_nsubj(doc)[0]
    token_t = relation_nsubj(doc)[1]
    if not is_auxiliar(doc):
        for i in doc:
            if ((i.dep_ == "advmod" and i.head.text == token_t) or 
               (i.dep_ == "adjmod" and i.head.text == token_t)  or 
               (i.dep_ == "advcl" and i.head.text == token_t) or 
                (i.dep_ == "amod" and i.head.text == token_t)):
                aspectos.append(token_h)
    return aspectos

## Regras 3 e 4

Referências: [1].

2 e 3, exemplos: Eu gosto das **lentes** da câmera, eu gosto da **beleza** da tela.

> Se t tem qualquer relação direta de objeto com um token n e o POS do token é Noun e n não está
no SenticNet, então n é extraído como um aspecto. Em (2), como está em relação direta de objeto com a lente
então a lente de aspecto é extraída.
(2)
Eu gosto da lente desta câmera.


> Se t tem qualquer relação direta de objeto com um token n e o POS do token n é Noun e n existe
no SenticNet, o token n é extraído como termo de aspecto. Na árvore de análise de dependência do
sentença, se outro token n 1 estiver conectado an usando qualquer relação de dependência e o POS de
n 1 é o substantivo, então n 1 é extraído como um aspecto. Em (3), como está em relação direta de objeto com a beleza
que está conectado à tela por meio de uma relação de preposição. Portanto, os aspectos da tela e da beleza são
extraído.
(3)
Gosto da beleza da tela.

In [66]:
def regra3_4(doc, aspectos):
    token_h = relation_nsubj(doc)[0]
    token_t = relation_nsubj(doc)[1]
    token_n = ""
    token_n1 = ""
    auxiliar = 0
    flag = 0
    pair = ("","")
    if not is_auxiliar(doc):
        for i in doc:
            if ((i.dep_ == "obj" or i.dep_ == "obj:pass") and 
                i.head.text == token_t):
                if i.pos_ == "NOUN":
                    token_n = i.text
                    try:
                        concept = bsn.concept(token_n)
                    except Exception as e: 
                        flag = 1
                        aspectos.append(token_n) 
                    if flag != 1:
                        sentiment = token_n 
                        pair = ("", sentiment)
        if not flag:
            for i in doc:
                if i.dep_ and i.head.text == token_n and i.pos_ == "NOUN":
                    flag = 1
                    token_n1 = i.text
                    aspect = token_n1
                    pair = (token_n1, sentiment)
                    aspectos.append(pair)                
                    break;
            if not flag:
                aspectos.append(pair)
    return aspectos

## Regra 5
Referências: [1].

4, exemplo: Eu gostaria de comentar sobre a **câmera** do celular.

> Se t está em relação de complemento de oração aberta com um token t1, então o aspecto t-t1 é extraído se t-t1
existe no léxico de opinião. Se t1 estiver conectado com um
token t2 cujo POS é Noun, então t2 é
extraído como um aspecto. Em (4), gosto e comentário estão na relação de complemento oracional e comentário
é conectado à câmera usando uma relação de preposição. Aqui, o POS da câmera é o substantivo e,
portanto, a câmera é extraída como um aspecto.
(4)
Eu gostaria de comentar sobre a câmera deste telefone.

In [67]:
def regra5(doc, aspectos):
    token_h = relation_nsubj(doc)[0]
    token_t = relation_nsubj(doc)[1]
    if not is_auxiliar(doc):
        for i in doc:
            if i.dep_ == "xcomp" and i.head.text == token_t:
                flag = 0
                token_t1 = i.text;
                try:
                    concept = bsn.concept(token_t1)
                except Exception as e:
                    flag = 0 
                finally:
                    flag = 1
                flag = 0
                try:
                    concept = bsn.concept(token_t1)
                except Exception as e:
                    flag = 0 
                finally:
                    flag = 1
                if flag:
                    ok = 0
                    for i in doc:
                        if i.head.text == token_t1:
                            token_h = i
                            ok = 1
                    if ok and token_h.pos_ == "NOUN":
                        aspectos.append(token_h)
                    elif (ok and token_h.pos_ != "NOUN"):
                        aspect = ' '.join([token_t, token_t1])
                        aspectos.append(aspect)
        return aspectos

## Regra 6

Referências: [1].

6, exemplo: A **câmera** é *boa*.

> Se o token t está em relação de cópula com um verbo copular e o POS de h é substantivo, então h é extraído
como um aspecto explícito. Em (6), a câmera é extraída como um aspecto.
(6) A câmera é boa.

In [68]:
def regra6(doc, aspectos):
    token_h, token_t = relation_nsubj(doc)
    for i in doc:
        if i.dep_ == "cop":
            for j in doc:
                if j.text == token_h:
                    if j.pos_ == "NOUN" or j.pos_ == "PROPN":
                        aspectos.append(token_h)
    return aspectos

##Regra 7
Referências: [1].

9, exemplo: Amo a **suavidade do jogador**.


> se um token h é conectado a um substantivo t usando uma relação preposicional, então h e t são extraídos como aspectos. Em (9) a suavidade é extraída como um aspecto. (9) Amo a suavidade do jogador.
>
> Regras Adicionais
>
> Um substantivo modificador composto de um NP é qualquer substantivo que serve para modificar o substantivo principal. Se t é extraído como um aspecto e t tem um modificador composto substantivo h, então o aspecto h-t é extraído e t é removido da lista de aspectos. Em (12), como frango e caçarola estão em uma relação de modificador composto substantivo, apenas a caçarola de frango é extraída como um aspecto. (12) Pedimos a caçarola de frango, mas o que encontramos foram alguns pedacinhos de frango, todo de carne escura e com osso.


In [69]:
def regra7(doc, aspectos):
    for i in doc:
        if (i.head.pos_ == "NOUN" or i.head.pos_ == "PROPN"):
            if (i.dep_ == "nmod" or i.dep_ == "amod" or i.dep_ == "acl" or i.dep_ == "acl:relcl"):
                token_h = i.text
                token_t = i.head.text
                if (i.dep_ == "nmod" and (i.head.pos_ == "NOUN" or i.head.pos_ == "PROPN")):
                    f = 0
                    for j in doc:
                        if (j.dep_ == "case" and j.head.text == token_h):
                            f = 1
                            aspect = ' '.join([token_t, j.text, token_h])
                            aspectos.append(aspect)
                            break
                    if not f:
                        aspect = ' '.join([token_t, token_h])
                        aspectos.append(aspect)
                else:
                    aspectos.append(token_t)
                break;
    return aspectos

## Regra 8
Referências: [1].

10, exemplo: Ana achou o **livro** *maravilhoso*.


> Se um token h está em uma relação direta de objeto com um token t, t é extraído como aspecto. Em (10), a menção está em
uma relação objetiva direta com o preço; portanto, o preço é extraído como um aspecto.
(10)
Sem falar no preço do telefone.

In [70]:
def regra8(doc, aspectos):
    token_t = ""
    token_h = ""
    for i in doc:
        if i.dep_ == "obj":
            token_t = i.text
            token_h = i.head.text
            aspectos.append(token_t)
            break;
    return aspectos

##Regra 9

Referências: [1].

11, exemplo: A **câmera** é *incrível* e *fácil de usar*.


> Para cada termo de aspecto extraído acima, se um termo de aspecto h estiver em coordenação ou relação conjunta
com outro token t, então t também é extraído como um aspecto. Em (11), surpreendente é primeiramente extraído como um
termo de aspecto. Como o incrível está em relação com o fácil, então o uso também é extraído como um aspecto.
(11)
A câmera é incrível e fácil de usar.

In [71]:
def regra9(doc, aspectos):
    token_h = relation_nsubj(doc)[0]
    token_t = relation_nsubj(doc)[1]
    sentiment1 = sentiment2 = ""
    for i in doc:
        if i.text == token_t:
            token_t1 = i.head.text
            if i.pos_ == "ADJ":
                sentiment1 = token_t1
    for i in doc:
        if i.dep_ == "conj" and i.head.text == sentiment1:
            sentiment2 = i.text
    for i in doc:
        if i.dep_ == "conj" and i.head.text == sentiment2:
            aspectos.append(i.text)
    return aspectos

# Regra 10
References: [2].

1, example: **vídeo** era *ruim*

> nsubj(JJ/OP,NN) 

In [72]:
def regra10(doc, aspectos):
    for NN in doc:
        if NN.dep_ == 'nsubj' and NN.head.pos_ == 'ADJ':
            aspectos.append(NN.text)
            break 
    return aspectos

#Regra 11
Referências: [2].

2, exemplo: a **qualidade** e a **lente** da câmera estão *aprovadas*.


>ReL1(H1,NN1) e ReL2(H1,NN2)
tal que ReL1 e ReL2 são quaisquer relações de dependência de
[‘nsubj’, ‘amod’, ‘prep’, ‘csubj’, ‘xsubj’, ‘dobj’, ‘iobj’]


In [73]:
def regra11(doc, aspectos):
    rel = ["nsubj", "amod", "prep", "csubj", "xsubj", "xsubj", "dobj", "iobj", "obj"]
    for i in doc:
        if i.dep_ == "conj" and i.head.dep_ in rel:
            if i.pos_ == "NOUN" and i.head.pos_ == "NOUN":
                aspectos.append(i.text)
                aspectos.append(i.head.text)
                return aspectos
    d = {}
    for i in doc:
        token_children = list(i.children)
        for j in token_children:
            if (j.dep_ in rel and (j.pos_ == "NOUN" or j.pos_ == "PROPN")):
                d[j.dep_] = (j.text, j.pos_)
    for i in d:
        aspectos.append(d[i][0])
    return aspectos

#Regra 12

Referências: [2].

3, exemplo: Honestamente, eu amo esse **jogador**.

> nsubj(VB1,H1) e dobj(VB1,NN)

In [74]:
def regra12(doc, aspectos):
    VB1 = relation_nsubj(doc)[1]
    for i in doc:
        if i.text == VB1 and i.pos_ == "VERB":
            for j in doc:
                if (j.dep_ == "obj" or j.dep == "dobj") and j.pos_ == "NOUN":
                    aspectos.append(j.text)
                    return aspectos

#Regra 13

Referências: [2].

4, exemplo: seu **tamanho** também faz dele *ideal* para viajar

> nsubj(H1, NN) e obj(H1, JJ/OP)
> nsubj(H1, NN) e ccomp(H1, JJ/OP)

In [75]:
def regra13(doc, aspectos):
    for i in doc:
        token_children = list(i.children)
        d = {} 
        for token in token_children:
            d[token.dep_] = (token.text, token.pos_) 
        if "nsubj" in d and "obj" in d:
            if d["nsubj"][1] == "NOUN":
                if d["obj"][1] == "ADJ":
                    aspectos.append(d["nsubj"][0])
                    return aspectos
                else:
                    if d["obj"][0] in op:
                        aspectos.append(d["nsubj"][0])
                        return aspectos

#Regra 14

Referências: [2].

5, exemplo: toca **dvds** e **cds** *originais*.

> amod(NN1,OP/JJ) e conj(NN1,NN2)


In [76]:
def regra14(doc, aspectos):
    for i in doc:
        if i.dep_ == "amod" and (i.pos_ == "ADJ" or i.text in op):
            j = i.head
            if j.pos_ == "NOUN" and j.dep_ == "conj" and j.head.pos_ == "NOUN":
                aspectos.append(j.text)
                aspectos.append(j.head.text)
                return aspectos 

#Regra 15

Referências: [2].

6, exemplo: acho a falta de **jogos** de entretenimento nesse celular um pouco *incômoda*

> nmod(OP/JJ,NNS)

In [77]:
def regra15(doc, aspectos):
    for i in doc:
        token = i.text
        if (i.dep_ == "nmod" and i.lemma_ != i.text and i.pos_ == "NOUN" 
            and token[len(token)-1].lower() == 's'):
            if (i.head.pos_ == "ADJ" or i.head.text in op):
                aspectos.append(i.text)
                return aspectos

#Regra 16

Referências: [2].

7, exemplo: o **manual** *escasso*

> amod(**NN**,OP)

In [78]:
def regra16(doc, aspectos):
    for i in doc:
        if (i.dep_ == "amod" and i.head.pos_ == "NOUN" and i.text in op):
            aspectos.append(i.head.text)
            return aspectos

#Regra 17

Referências: [2].

8, exemplo: esta **câmera** tem uma grande falha de design


> Rel1(H1,NN) e Rel2(H1, OP/JJ)
tal que ReL1 e ReL2 são quaisquer relações de dependência de
[‘nsubj’, ‘csubj’, ‘xsubj’, ‘dobj’, ‘iobj’]

In [79]:
def regra17(doc, aspectos):
    for i in doc:
        children_list = list(i.children)
        dep_set = ["nsubj", "csubj", "xsubj", "dobj", "iobj", "obj"]
        d = {}
        for token in children_list:
            if token.dep_ in dep_set:
                d[token.dep_] = (token.text, token.pos_)
        if len(d) >= 2:
            REL2 = 0
            for i in d:
                if d[i][1] == "ADJ" or d[i][0] in op:
                    REL2 = 1
            if REL2:
                for i in d:
                    if d[i][1] == "NOUN":
                        aspectos.append(d[i][0])
                        return aspectos

#Regra 18
Referências: [2].

9, exemplo: minha única reclamação sobre o hardware são os **botões**


> nsubj(NN,OP/JJ)

In [80]:
def regra18(doc, aspectos):
    for i in doc:
        if i.dep_ == "nsubj" and i.head.pos_ == "NOUN":
            if (i.pos_ == "ADJ" or i.text in op):
                aspectos.append(i.head.text)
                return aspectos

#Regra 19

Referências: [2].

10, exemplo: Eu gostei especialmente dos **botões** mais comumente usados

> dobj(OP/JJ,NN) 
> 
> O POS de "gosto de" não está com sua tag correta

In [81]:
def regra19(doc, aspectos):
    for i in doc:
        if i.pos_ == "NOUN" and (i.dep_ == "obj" or i.dep_ == "dobj" or i.dep_ == "iobj"):
            if (i.head.text in op or i.head.pos_ == "ADJ"):
                aspectos.append(i.text)
                return aspectos
        elif i.pos_ == "NOUN" and i.dep_ == "nmod":
            if i.head.text in op:
                aspectos.append(i.text)
                return aspectos

#Regra 20

Referências: [2].

11, exemplo: eu acho **exibições na tela** *irritantes*
> obj(OP/JJ,NN1) e nmod(NN1,NN2)


In [82]:
def regra20(doc, aspectos):
    for i in doc:
        if i.dep_ == "nmod" and i.pos_ == "NOUN":
            if i.head.dep_ == "obj" and i.head.pos_ == "NOUN":
                if i.head.nbor().pos_ == "ADP":
                    aspectos.append(i.head.text+' '+i.head.nbor().text+' '+i.text)
                    return aspectos

#Regra 21

References: [2].

12, example: **qualidade** e **lentes** da câmera *comprovadas*

> conj(NN1,NN2) 


In [83]:
def regra21(doc, aspectos):
    for i in doc: 
        if i.dep_ == "conj" and i.pos_ == "NOUN" and i.head.pos_ == "NOUN":
            j = i.head
            aspectos.append(j.text)
            aspectos.append(i.text)
            return aspectos

# Regra 22

Referências: [2].

13, exemplo: No geral, o g3 entrega o que deve ser considerado a *melhor* **qualidade de imagem**.

> amod(NN1,OP/JJ) e nmod(NN1,NN2)


In [84]:
def regra22(doc, aspectos):
    for i in doc:
        if i.dep_ == "case":
            j = i.head
            if j.dep_ == "nmod" and j.pos_ == "NOUN":
                k = j.head
                token_children = list(k.children)
                for l in token_children:
                    if l.dep_ == "amod":
                        aspectos.append(k.text+" "+i.text+" "+j.text)
                        return aspectos

#Regra 23

Referências: [2].

14, exemplo: As *cores* na tela não são tão *nítidas* quanto eu gostaria que fossem.

> neg(OP/JJ, H1) e nsubj(OP/JJ,NN)


In [85]:
def regra23(doc, aspectos):
    for i in doc: 
        token_children = list(i.children)
        dep_children = []
        for j in token_children:
            dep_children.append(j.dep_)
        if ("advmod" in dep_children or "neg" in dep_children) and ("nsubj" in 
            dep_children or "nsubj:pass" in dep_children) and (i.pos_ == "ADJ" or i.text in op):
            for k in token_children:
                if k.pos_ == "NOUN" and k.dep_ == "nsubj":
                    aspectos.append(k.text)
                    return aspectos 

#Regra 24

Referências: [2].

15, exemplo: definitivamente, uma *boa* **câmera**

> ReL(NN, OP/JJ)
tal que ReL é qualquer relação de dependência de
[‘nsubj’, ‘amod’, ‘prep’, ‘csubj’, ‘xsubj’, ‘dobj’, ‘iobj’]


In [86]:
def regra24(doc, aspectos):
    for i in doc:
        rel = ["nsubj", "amod", "prep", "csubj", "xsubj", "dobj", "iobj", "obj"]
        if i.dep_ in rel:
            if (i.head.text in op or i.head.pos_ == "ADJ") and i.pos_ == "NOUN":
                aspectos.append(i.text)
                return aspectos
            elif (i.pos_ == "ADJ" or i.text in op) and i.head.pos_ == "NOUN":
                aspectos.append(i.head.text)
                return aspectos

#Regra 25

References: [2].

17, exemplo: Os **cardápios** são *fáceis* de navegar.


> nsubj(OP1/JJ1,NN) e cop (OP1/JJ1,H1) 


In [87]:
def regra25(doc, aspectos):
    for i in doc: 
        token_children = list(i.children)
        dep_children = []
        for j in token_children:
            dep_children.append(j.dep_)
        if "nsubj" in dep_children and "cop" in dep_children:
            for k in token_children:
                if k.pos_ == "NOUN" or k.pos_ == "PROPN":
                    aspectos.append(k.text)
                    return aspectos

#Regra 26

References: [2], tabela 4.

18, exemplo: Os **controles** são *feios*


> **NNS** VBP OP/JJ 

In [88]:
def regra26(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.text[len(i.text)-1] == 's' and i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif (i.pos_ == "AUX" or i.pos_ == "VERB") and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 27

References: [2], tabela 4.

19, exemplo: O **áudio** é *excelente*


> **NN** VBZ JJ/OP

In [89]:
def regra27(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif (i.pos_ == "AUX" or i.pos_ == "VERB") and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos


#Regra 28 -> 7

References: [2], tabela 4.

20, exemplo: É um **tocador de dvd** muito *bom*


> **NN** **NN** JJ/OP

In [90]:
def regra28(doc, aspectos):  
    return

REGRA ADAPTADA
#Regra 29

References: [2], tabela 4.

21, exemplo: é um **produto** muito *incrível*

> **NN** RB JJ/OP

In [91]:
def regra29(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            val = max(val, 1)
            aspecto = i.text
        elif i.pos_ == "ADV" and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 30

References: [2], tabela 4.

22, exemplo: *baixa* **confiabilidade**
> JJ/OP **NN**

In [92]:
def regra30(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if (i.pos_ == "ADJ" or i.text in op) and val == 0:
            val = max(val, 1)
        elif i.pos_ == "NOUN" and val == 1:
            # if val <= 1: aspecto = i.text
            aspecto = i.text
            val = max(val, 2)
            break
        else: val = 0
    if(val == 2 and aspecto != ""):
        aspectos.append(aspecto)
    return aspectos

#Regra 31

References: [2], tabela 4.

23, exemplo: **software** criativo *complexo*
> **NN** JJ/OP

In [93]:
def regra31(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if(i.pos_ == "NOUN") and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 1:
            val = max(val, 2)
            break
        else: val = 0
    if(val == 2 and aspecto != ""):
        aspectos.append(aspecto)
    return aspectos

#Regra 32

References: [2], tabela 4.

24, exemplo: **áudio** no vídeo também faltando
> **NN** IN NN

In [94]:
def regra32(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if (i.pos_ == "NOUN" or i.pos_ == "PROPN") and val == 0:
            val = max(val, 1)
            aspecto = i.text
        elif i.pos_ == "ADP" and val == 1:
            val = max(val, 2)
        elif i.pos_ == "NOUN" and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 33

References: [2], tabela 4.

25, exemplo: a **construção** do player é a mais *cafona* que eu já vi

> **NN** IN DT NN

In [95]:
def regra33(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif i.pos_ == "ADP" and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "PROPN" or i.pos_ == "NOUN") and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 34

References: [2], tabela 4.

26, exemplo: no geral, uma *boa* compra para o **preço**

> NN IN DT **NN**

In [96]:
def regra34(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            val = max(val, 1)
        elif i.pos_ == "ADP" and val == 1:
            val = max(val, 2)
        elif i.pos_ == "DET" and val == 2:
            val = max(val, 3)
        elif i.pos_ == "NOUN" and val == 3:
            val = max(val, 4)
            if(val == 4): 
                aspecto = i.text
                break
        else: val = 0
    if val == 4 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 35

References: [2], tabela 4.

27, exemplo: ele recusou a **ler** discos secundários

> OP + preposição "a" + **VB**

In [97]:
def regra35(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.text in op and val == 0:
            val = max(val, 1)
        elif (i.pos_ == "SCONJ" or i.pos_ == "PREP") and val == 1:
            val = max(val, 2)
        elif i.pos_ == "VERB" and val == 2:
            if val <= 3: aspecto = i.text
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 36

References: [2], tabela 4.

28, exemplo: é extremamente *simples* de **navegar**
> JJ + preposição "para" + **VB** tal que JJ não está em OP

In [98]:
def regra36(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if (i.pos_ == "ADJ" and i.text not in op) and val == 0:
            val = max(val, 1)
        elif i.pos_ == "SCONJ" or i.pos_ == "PREP":
            val = max(val, 2)
        elif i.pos_ == "VERB" and val == 2:
            if val <= 3: aspecto = i.text
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 37

References: [2], tabela 4.

29, exemplo: o g3 tem **deslocamentos brancos** muito mais *nítidos*
> **NN JJ1** JJ2, tal que JJ2 não está em op

In [99]:
def regra37(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif i.pos_ == "ADJ" and val == 1 and (i.text not in op):
            if val <= 2: aspecto += ' ' + (i.text)
            val = max(val, 2)
        elif i.pos_ == "ADJ" and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 38

References: [2], tabela 4.

30, exemplo: o **manual** faz um *bom* trabalho

> **NN** VBZ/VBP DT OP/JJ NN

In [100]:
def regra38(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif i.pos_ == "VERB" and val == 1:
            val = max(val, 2)
        elif i.pos_ == "DET" and val == 2:
            val = max(val, 3)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 3:
            val = max(val, 4)
        elif i.pos_ == "NOUN" and val == 4:
            val = max(val, 5)
            break
        else: val = 0
    if val == 5 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 39

References: [2], tabela 4.

31, exemplo: **apex** é a *melhor* marca de qualidade barata para leitores de DVD

> **NN** * PRP/DT OP, tal que * é qualquer padrão encontrado que não seja NN

In [101]:
def regra39(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" or i.pos_ == "PROPN":
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif i.pos_ != "NOUN" and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "DET" or i.tag == "PRP") and val == 2:
            val = max(val, 3)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 3:
            val = max(val, 4)
            break
        else: val = 0
    if val == 4 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 40

References: [2], tabela 4.

32, exemplo: tem ótima recepção

> VB OP/JJ **NN**

In [102]:
def regra40(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "VERB" and val == 0:
            val = max(val, 1)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 1:
            val = max(val, 2)
        elif i.pos_ == "NOUN" and val == 2:
            if val <= 3: aspecto = i.text
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 41 -> 27

References: [2], tabela 4.

33, exemplo: áudio é excelente

> **NN** VB OP/JJ 

In [103]:
def regra41(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if i.pos_ == "NOUN" and val == 0:
            if val <= 1: aspecto = i.text
            val = max(val, 1)
        elif (i.pos_ == "VERB" or i.pos_ == "AUX") and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "ADJ" or i.text in op) and val == 2:
            val = max(val, 3)
            break
        else: val = 0
    if val == 3 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

# Regra 42

References: [2], tabela 5.

regra 51, exemplo: a tecnologia **mms** é muito bem integrada.

> **NN** NN VBZ/VBP RB RB

In [104]:
def regra42(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if (i.pos_ == "PROPN" or i.pos_ == "NOUN") and val == 0:
            val = max(val, 1)
        elif (i.pos_ == "PROPN" or i.pos_ == "NOUN") and i.dep_ == "flat:name"  and val == 1:
            val = max(val, 2)
            aspecto = i.text
        elif (i.pos_ == "VERB" or i.pos_ == "AUX")and val == 2:
            val = max(val, 3)
        elif i.pos_ == "ADV" and val == 3:
            val = max(val, 4)
        elif i.pos_ == "ADV" and val == 4:
            val = max(val, 5)
            break
        else: val = 0
    if val == 5 and aspecto != "":
        aspectos.append(aspecto)
    return aspectos

#Regra 43
References: [2], tabela 5.

regra 80, exemplo: **controles** e **opções** são fáceis

> **NN** CC **NN** VBP JJ/OP

In [105]:
def regra43(doc, aspectos):
    val = 0
    aspecto1 = ""
    aspecto2 = ""
    for i in doc:
        if (i.pos_ == "NOUN" or i.pos_ == "PROPN")and val == 0:
            val = max(val, 1)
            aspecto1 = i.text
        elif i.pos_ == "CCONJ" and val == 1:
            val = max(val, 2)
        elif (i.pos_ == "NOUN" or i.pos_ == "PROPN") and val == 2:
            val = max(val, 3)
            aspecto2 = i.text
        elif (i.pos_ == "VERB" or i.pos_ == "AUX") and val == 3:
            val = max(val, 4)
        elif i.pos_ == "ADJ" or i.pos_ in op:
            val = max(val, 5)
            break
        else:
            val = 0
    if val == 5 and aspecto1 != "" and aspecto2 != "":
        aspectos.append(aspecto1)
        aspectos.append(aspecto2)

    return aspectos

#Regra 44
References: [2], tabela 5.

regra 103, exemplo: **recurso sunset **bate fotos incríveis

> **NN** **NN** * JJ/OP NN

In [106]:
def regra44(doc, aspectos):
    val = 0
    aspecto = ""
    for i in doc:
        if (i.pos_ == "NOUN" or i.pos_ == "PROPN") and val == 0:
            val = max(val, 1)
            aspecto = i.text
        elif (i.pos_ == "NOUN" or i.pos_ == "PROPN") and val == 1:
            val = max(val, 2)
            aspecto+= ' '+i.text
        elif i.pos_ != "NOUN" and val == 2:
            val = max(val, 3)
        elif i.pos_ == "NOUN" and val == 3:
            val = max(val, 4)
        elif (i.pos_ == "ADJ" or i.pos_ in op) and val == 4:
            val = max(val, 5)
            break
        else: val = 0
    if val == 5 and aspecto:
        aspectos.append(aspecto)
    return aspectos

#Regra 45
References: [2], tabela 5.

regra 120, exemplo: a canon g3 é a melhor **câmera digital**
> amod(NN1,JJ/OP) and amod(**NN1**,**JJ**) [such that JJ not in OP]

In [107]:
def regra45(doc, aspectos):
    f = 0
    cont = 0
    aspecto = ""
    for i in doc:
        if i.dep_ == "amod":
            cont+=1
        
        if i.dep_ == "amod" and cont > 1:
            if (i.text not in op):
                aspecto = i.head.text+' '+i.text
    aspectos.append(aspecto)
    return aspectos

#Conjunto de pares <sentença-aspecto>
* gera todos os pares de sentença-aspecto do dataset


In [108]:
def gerarConjuntoPares(file):
    if(glob.glob("restaurantes.xml")):
        tree = ET.parse(file[0])
        print(tree)
        root = tree.getroot()
        par = ("","")
        conjunto_de_pares = set()
        f = 0
        for review in root.iter('review'):
            for x in review.iter():
                if x.tag == 'sentence':
                    if f == 1:
                        par = (frase, "")
                    frase = x.text
                    f = 1
                if x.tag == 'opinion':
                    f = 0
                    par = (frase, x.attrib['aspect'])
                    conjunto_de_pares.add(par)
    if file == glob.glob('reli.csv') or file == glob.glob('rehol.csv'):
        df = pd.read_csv('reli.csv')
        conjunto_de_pares = set()
        for key in df.itertuples():
            par = ("", "")
            # transformando em lista de tuplas
            lista = eval( "[%s]" % key[3])
            for tupla in lista:
                # a sentença está nessa posição
                sentenca = key[2]
                # split nas sentenças da string
                sent = sentenca.split()
                for j in tupla:
                    # caso haja apenas um aspecto
                    if(len(j[0]) == 1): 
                        index = j[0][0]
                        aspecto = sent[index]
                        par = (sentenca, aspecto)
                    # caso haja mais de um aspecto na sentenca
                    else:
                        aux = []
                        aspecto = ""
                        for k in j[0]:
                            index = k 
                            aux.append(sent[index])
                            aspecto = ' '.join(aux)
                        par = (sentenca, aspecto)
                    conjunto_de_pares.add(par)
    if(glob.glob("train.csv")):
        df = pd.read_csv('train.csv', on_bad_lines='skip', sep=";" )
        reviews = list(df['review'])
        aspects = list(df['aspect'])
        conjunto_de_pares = set()
        for r, a in zip(reviews, aspects):
            par = (r, a)
            conjunto_de_pares.add(par)
    return conjunto_de_pares

# Extração de Aspectos
* extração por meio das regras implementadas

* retorna um set de aspectos

In [109]:
def extrairAspectos(sentencas, regra):
    aspectos = []
    for sentenca in sentencas:
        doc = nlp(sentenca[0])
        regra(doc, aspectos)
    return aspectos

# Métricas de Avaliação

* aplicando as métricas de avaliação 

In [110]:
def gerarMetricas(aspectos_extraidos, sentencas):

    # Precisao: 
    elementos = 0
    for a in aspectos_extraidos:
        for g in sentencas:
            if str(a) == g[1]:
                elementos+=1
                break
    # print(len(aspectos_extraidos))
    if len(aspectos_extraidos) == 0:
        precisao = 0
    else: precisao = elementos/len(aspectos_extraidos)
    # print(f"Precisão: {precisao} = {elementos} / {len(aspectos)}")

    # Revocacao
    elementos = 0
    for g in sentencas:
        for a in aspectos_extraidos:
            if str(a) == g[1]:
                elementos+=1
                break
    if len(sentencas) == 0: 
        revocacao = 0
    else: 
        revocacao = elementos/len(sentencas)
    
    # print(f"Revocação: {revocacao} = {elementos} / {len(aspectosGabarito)}\n")

    # F1-score
    if precisao * revocacao == 0: 
        f1 = 0
    else: 
        f1 = 2 * precisao * revocacao / (precisao + revocacao)
    
    return (precisao, revocacao, f1)

# Regras Por ID

identificar cada regra de acordo com seu ID

In [111]:
regraPorId = {
    '1': regra1,
    '2': regra2,
    '3_4': regra3_4,
    '5': regra5,
    '6': regra6,
    '7': regra7,
    '8': regra8,
    '9': regra9,
    '10': regra10,
    '11': regra11,
    '12': regra12,
    '13': regra13,
    '14': regra14,
    '15': regra15,
    '16': regra16,
    '17': regra17,
    '18': regra18,
    '19': regra19,
    '20': regra20,
    '21': regra21,
    '22': regra22,
    '23': regra23,
    '24': regra24,
    '25': regra25,
    '26': regra26,
    '27': regra27,
    '28': regra28,
    '29': regra29,
    '30': regra30,
    '31': regra31,
    '32': regra32,
    '33': regra33,
    '34': regra34,
    '35': regra35,
    '36': regra36,
    '37': regra37,
    '38': regra38,
    '39': regra39,
    '40': regra40,
    '41': regra41,
    '42': regra42,
    '43': regra43,
    '44': regra44,
    '45': regra45
}

# Funções para Gerar os Resultados


In [112]:
class Metrica(Enum):
    PRECISAO = 0
    REVOCACAO = 1
    F1 = 2

def ordenaResultados(resultados, metrica1 = Metrica.PRECISAO, metrica2 = Metrica.REVOCACAO, metrica3 = Metrica.F1):
    '''
    Parâmetros:
    resultados: 
        Dicionário contendo o índice da regra como chave e uma tupla de métricas
        como valor
    metrica: 
        Métrica por qual os resultados devem ser ordenados

    Retorna:
    Lista com os mesmos itens do dicionário recebido ordenada de forma
    decrescente pela métrica escolhida
    '''
    return sorted(
        resultados.items(),
        key=lambda item: (item[1][metrica1.value], item[1][metrica2.value], item[1][metrica3.value]),
        reverse=True)

def geraResultadosPorRegra():
    """
    Retorna:
        Dicionário contendo o índice da regra como chave e uma tupla de métricas
        como valor
    """
    if(glob.glob("*.xml")):
        sentencas = gerarConjuntoPares(glob.glob("*.xml"))
    if(glob.glob("*.csv")):
        sentencas = gerarConjuntoPares(glob.glob("*.csv"))
    if(glob.glob("*.json")):
        sentencas = gerarConjuntoPares(glob.glob("*.json"))
    resultados = {}

    for id in regraPorId:
        aspectos_extraidos = extrairAspectos(sentencas, regraPorId[id])
        # print(aspectos_extraidos)
        resultados[id] = gerarMetricas(aspectos_extraidos, sentencas)
    return resultados

#Algoritmo Guloso

In [ ]:
def guloso():
    resultados = ordenaResultados(geraResultadosPorRegra(), Metrica.REVOCACAO, Metrica.PRECISAO, Metrica.F1)
    for r in resultados:
        print(r)
    conjuntoRegras = []
    # Obtemos a regra com melhor precisão
    melhorPrecisao = resultados[0]
    # Adicionamos a função que contém a regra ao conjunto ótimo de regras
    conjuntoRegras.append(regraPorId[melhorPrecisao[0]])
    # Removemos a tupla do conjunto a ser testado
    del resultados[0]
    melhorF1 = -1
    melhorR = -1
    if(glob.glob("*.xml")):
        sentencas = gerarConjuntoPares(glob.glob("*.xml"))
    if(glob.glob("*.csv")):
        sentencas = gerarConjuntoPares(glob.glob("*.csv"))
    if(glob.glob("*.json")):
        sentencas = gerarConjuntoPares(glob.glob("*.json"))
    
    # lista de aspectos extraídos
    conjunto_aspectos = extrairAspectos(sentencas, regraPorId[melhorPrecisao[0]])

    while True:
        regraOtima = None
        # conjunto_aspectos = []
        aux = []
        for tupla in resultados:
            regra = regraPorId[tupla[0]]
            conjuntoRegras.append(regra)
            aspectos_extraidos = extrairAspectos(sentencas, regra)
            resultado = gerarMetricas(conjunto_aspectos + aspectos_extraidos, sentencas)
            print("\nCom a regra {}: \n P: {} \n R: {} \n F1: {}"
            .format( tupla[0], resultado[0], resultado[1], resultado[2]))
            if resultado[2] > melhorF1 or (resultado[2] == melhorF1 and 
                                            (resultado[0] > melhorP or resultado[1] > melhorR)):
                melhorP = resultado[0]
                melhorR = resultado[1]
                melhorF1 = resultado[2]
                regraOtima = regra
                tuplaOtima = tupla
                aux = aspectos_extraidos
            
            conjuntoRegras.remove(regra)
        if regraOtima != None:
            print("\nREGRA ADICIONADA: {}\n".format(tuplaOtima[0]))
            conjuntoRegras.append(regraOtima)
            conjunto_aspectos += aux
            del resultados[resultados.index(tuplaOtima)]
        else: break
    return conjuntoRegras

g = guloso()
for i in g:
    print(i)

('11', (0.5224116161616161, 0.7851072440307568, 0.6273701385140334))
('25', (0.6994475138121546, 0.745447187373533, 0.721715127695903))
('6', (0.68389662027833, 0.745447187373533, 0.7133466550336327))
('10', (0.7666666666666667, 0.7401861594496155, 0.753193736963197))
('8', (0.36858190709046457, 0.736139214892756, 0.4912146428813741))
('7', (0.42843646669815777, 0.7316875758802105, 0.540427796049027))
('24', (0.680695847362514, 0.6972885471469041, 0.6888922985594728))
('31', (0.6255044390637611, 0.6847430190206394, 0.6537845891210902))
('21', (0.4080675422138837, 0.679481991096722, 0.509906974519836))
('12', (0.447242206235012, 0.6475111290975314, 0.5290585497076128))
('39', (0.4025157232704403, 0.641845406717928, 0.49475772449661))
('16', (0.6707152496626181, 0.6390125455281263, 0.6544802066280087))
('14', (0.4859154929577465, 0.6349656009712666, 0.5505305151069039))
('34', (0.46814404432132967, 0.6325374342371509, 0.5380641691840439))
('27', (0.8193384223918575, 0.6305139619587211, 0

In [ ]:
sentencas = gerarConjuntoPares(glob.glob("*.csv"))
conjuntoRegras = [regra11, regra6, regra27, regra41, regra23, regra13, regra38]
aspectos_extraidos = []
for regra in conjuntoRegras:
    aspectos_extraidos += extrairAspectos(sentencas, regra)
print(gerarMetricas(aspectos_extraidos, sentencas))

# REFERENCES

[1] PORIA, Soujanya et al. A rule-based approach to aspect extraction from product reviews. In: Proceedings of the second workshop on natural language processing for social media (SocialNLP). 2014. p. 28-37.

[2] M. Tubishat, N. Idris, and M. Abushariah, ‘‘Explicit aspects extraction in sentiment analysis using optimal rules combination,’’ Future Gener. Comput. Syst., vol. 114, pp. 448–480, Jan. 2021

[3] Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
;       Proceedings of the ACM SIGKDD International Conference on Knowledge 
;       Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
;       Washington, USA